In [1]:
from crewai import Agent,Crew
from tools.browser_tools import BrowserTools
from tools.calculator_tools import CalculatorTools
from tools.search_tools import SearchTools

In [2]:
# load SERPER_API_KEY
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_community.llms import Ollama
llm = Ollama(
    model="llama3.2:latest",
    temperature=0,
    # other params...
)

In [4]:
from trip_tasks import TripTasks
tasks = TripTasks()

### city_selection_agent

In [5]:
city_selector_agent = Agent(
        role='City Selection Expert',
        goal='Select the best city based on weather, season, and prices',
        backstory=
        'An expert in analyzing travel data to pick ideal destinations',
        tools=[
            SearchTools.search_internet,
            BrowserTools.scrape_and_summarize_website,
        ],
        verbose=True,
        llm =llm)


In [6]:
local_expert_agent = Agent(
        role='Local Expert at this city',
        goal='Provide the BEST insights about the selected city',
        backstory="""A knowledgeable local guide with extensive information
        about the city, it's attractions and customs""",
        tools=[
            SearchTools.search_internet,
            BrowserTools.scrape_and_summarize_website,
        ],
        verbose=True,
        llm =llm)

In [7]:
travel_concierge_agent = Agent(
        role='Amazing Travel Concierge',
        goal="""Create the most amazing travel itineraries with budget and 
        packing suggestions for the city""",
        backstory="""Specialist in travel planning and logistics with 
        decades of experience""",
        tools=[
            SearchTools.search_internet,
            BrowserTools.scrape_and_summarize_website,
            CalculatorTools.calculate,
        ],
        verbose=True,
        llm =llm)

In [18]:
location = "Bangkok"
cities = "Berlin or Japan"
date_range = "New year holidays"
interests = "AI , Latest Technology and nature "

In [19]:
identify_task = tasks.identify_task(
      city_selector_agent,
      location,
      cities,
      interests,
      date_range)

In [20]:
gather_task = tasks.gather_task(
  local_expert_agent,
  location,
  interests,
  date_range
)

In [21]:
plan_task = tasks.plan_task(
   travel_concierge_agent, 
   location,
   interests,
   date_range
   )

In [22]:
crew = Crew(
    agents=[
      city_selector_agent, local_expert_agent, travel_concierge_agent
    ],
    tasks=[identify_task, gather_task, plan_task],
    verbose=True
  )


2024-11-22 23:53:43,309 - 8563341376 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [23]:

result = crew.kickoff()

 [DEBUG]: == Working Agent: City Selection Expert
 [INFO]: == Starting Task: 
Analyze and select the best city for the trip based 
on specific criteria such as weather patterns, seasonal
events, and travel costs. This task involves comparing
multiple cities, considering factors like current weather
conditions, upcoming cultural or seasonal events, and
overall travel expenses. 

Your final answer must be a detailed
report on the chosen city, and everything you found out
about it, including the actual flight costs, weather 
forecast and attractions.
If you do your BEST WORK, I'll tip you $100!

Traveling from: Bangkok
City Options: Berlin or Japan
Trip Date: New year holidays
Traveler Interests: AI , Latest Technology and nature 



> Entering new CrewAgentExecutor chain...
Action: Delegate work to co-worker
Action Input: {"coworker": "Local Expert at this city", "task": "Provide information about Berlin and Japan cities", "context": "Comparison of weather patterns, seasonal events, and 

In [26]:
print(result)

Agent stopped due to iteration limit or time limit.


In [25]:
print(len(result))

51
